![dvd_image](dvd_image.jpg)

A DVD rental company needs your help! They want to figure out how many days a customer will rent a DVD for based on some features and has approached you for help. They want you to try out some regression models which will help predict the number of days a customer will rent a DVD for. The company wants a model which yeilds a MSE of 3 or less on a test set. The model you make will help the company become more efficient inventory planning.

The data they provided is in the csv file `rental_info.csv`. It has the following features:
- `"rental_date"`: The date (and time) the customer rents the DVD.
- `"return_date"`: The date (and time) the customer returns the DVD.
- `"amount"`: The amount paid by the customer for renting the DVD.
- `"amount_2"`: The square of `"amount"`.
- `"rental_rate"`: The rate at which the DVD is rented for.
- `"rental_rate_2"`: The square of `"rental_rate"`.
- `"release_year"`: The year the movie being rented was released.
- `"length"`: Lenght of the movie being rented, in minuites.
- `"length_2"`: The square of `"length"`.
- `"replacement_cost"`: The amount it will cost the company to replace the DVD.
- `"special_features"`: Any special features, for example trailers/deleted scenes that the DVD also has.
- `"NC-17"`, `"PG"`, `"PG-13"`, `"R"`: These columns are dummy variables of the rating of the movie. It takes the value 1 if the move is rated as the column name and 0 otherwise. For your convinience, the reference dummy has already been dropped.

In [78]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Import any additional modules and start coding below

In [79]:
df = pd.read_csv('rental_info.csv')

print(df.columns)

df['return_date'] = pd.to_datetime(df['return_date'])
df['rental_date'] = pd.to_datetime(df['rental_date'])

df['rental_length'] = df['return_date'] - df['rental_date']

df['rental_length_days'] = df['rental_length'].dt.days

Index(['rental_date', 'return_date', 'amount', 'release_year', 'rental_rate',
       'length', 'replacement_cost', 'special_features', 'NC-17', 'PG',
       'PG-13', 'R', 'amount_2', 'length_2', 'rental_rate_2'],
      dtype='object')


In [80]:
df["deleted_scenes"] =  np.where(df["special_features"].str.contains("Deleted Scenes"), 1,0)
df['behind_the_scenes'] = np.where(df["special_features"].str.contains("Behind the Scenes"), 1,0)

In [81]:
print(df.columns)
print(df.info())

Index(['rental_date', 'return_date', 'amount', 'release_year', 'rental_rate',
       'length', 'replacement_cost', 'special_features', 'NC-17', 'PG',
       'PG-13', 'R', 'amount_2', 'length_2', 'rental_rate_2', 'rental_length',
       'rental_length_days', 'deleted_scenes', 'behind_the_scenes'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15861 entries, 0 to 15860
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype              
---  ------              --------------  -----              
 0   rental_date         15861 non-null  datetime64[ns, UTC]
 1   return_date         15861 non-null  datetime64[ns, UTC]
 2   amount              15861 non-null  float64            
 3   release_year        15861 non-null  float64            
 4   rental_rate         15861 non-null  float64            
 5   length              15861 non-null  float64            
 6   replacement_cost    15861 non-null  float64            
 7   special_features 

In [97]:
df_int = df.drop(columns=['rental_date', 'return_date', 'special_features', 'rental_length', 'rental_length_days'])

In [99]:
X = df_int

y = df['rental_length_days']

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, stratify = y, random_state =1)


In [84]:
from sklearn.linear_model import Lasso

model = Lasso(alpha = 0.3, random_state = 9)

model.fit(X_train, y_train)

lasso_coef = model.coef_
print(coef)

X_lasso_train, X_lasso_test = X_train.iloc[:, lasso_coef > 0], X_test.iloc[:, lasso_coef > 0]

[ 5.79320139e-01 -0.00000000e+00 -0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
 -0.00000000e+00  4.41899231e-02  2.95688704e-06 -1.53331040e-01
 -0.00000000e+00  0.00000000e+00]


In [85]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

linear_model = LinearRegression()
linear_model.fit(X_lasso_train, y_train)
y_pred_lasso = linear_model.predict(X_lasso_test)
mse_lin_reg_lasso = mean_squared_error(y_test, y_pred_lasso)

print(mse_lin_reg_lasso)


4.719178905535936


In [101]:

dtr_model = DecisionTreeRegressor(random_state=42)
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error, r2_score

param_grid = {'max_depth': np.arange(1,11,1)}

random_search = RandomizedSearchCV(
    estimator=dtr_model,
    param_distributions=param_grid,
    n_iter=30,          # Try only 30 combinations
    scoring='r2',
    cv=5,
    random_state=42,
    n_jobs=-1
)

random_search.fit(X_train, y_train)

best_model = random_search.best_estimator_

y_pred_dtr = best_model.predict(X_test)


mse_dtr = mean_squared_error(y_test, y_pred_dtr)

print(mse_dtr)


2.344730954627001


In [102]:



rfr_model = RandomForestRegressor(random_state=42)


param_grid = {
    'max_depth': np.arange(1,11,1),
    'n_estimators': np.arange(1,101,1)
}

random_search_rfr = RandomizedSearchCV(
    estimator=rfr_model,
    param_distributions=param_grid,
    n_iter=30,          # Try only 30 combinations
    scoring='r2',
    cv=5,
    random_state=42,
    n_jobs=-1
)

random_search_rfr.fit(X_train, y_train)

best_model_rfr = random_search_rfr.best_estimator_

y_pred_rfr = best_model_rfr.predict(X_test)


mse_rfr = mean_squared_error(y_test, y_pred_rfr)

print(mse_rfr)


2.1387946002611473


In [88]:
best_model = rfr
best_mse = mse_rfr